# ABM_Tutorial_Wellcome_EBEC
### File: 1_notebook_simulation_with_classes_and_functions_add_lineage_ids.ipynb
### Date: 2025.06.30

### Description

- run simulation in a smaller lattice
    - using user-defined parameters

- analyse tumour sizes using DBSCAN

- this contains a simple example of phenotypic heterogeneity (survival), with parameters selected to demonstrate extinction of certain clones (lineage id == even number) while expansion of other clones

### Note for Google Colab

- files.upload() to load .json and .csv files

# Library

In [ ]:
# set up environment ...

In [4]:
import json
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.cluster import DBSCAN

In [5]:
from typing import Tuple, Dict

# Classes and functions

In [6]:
class Cell:
    def __init__(self, cell_attributes: Dict):
        self.attributes = cell_attributes

    def get_attributes(self) -> Dict:
        return self.attributes

    def set_attributes(self, updated_attributes: Dict):
        self.attributes = updated_attributes

class CancerCell(Cell):
    def __init__(self, cell_attributes: Dict):
        super().__init__(cell_attributes)

class Hepatocyte(Cell):
    def __init__(self, cell_attributes: Dict):
        super().__init__(cell_attributes)

In [7]:
def get_cell_configurations() -> Tuple[Dict]:
    """_summary_

    Returns:
        Tuple[Dict]: A Tuple of Dict variables, including
            site_types: Dict[int, str] maps integer to string names of site types
            sites_states: Dict[int, str] maps integer to string names of site states
            color_map: Dict[str, str] contains the colors corresponding to site types
            markersize_map: Dict[str, float] contains the marker sizes corresponding to site types
    """

    # configuration for cells
    site_types = {
        0: "CV", # central vein
        1: "PT", # portal triad
        2: "HEP", # hepatocyte
        3: "NO", # not occupied
        4: "CC", # cancer cell
        5: "ECM"
    }

    sites_states = {
        0: "quiescent",
        1: "proliferative",
        2: "apoptotic",
        # 3: "migratory"
    }

    color_map = {
        "CV" : "blue",
        "PT": "red",
        "HEP": "lightgreen",
        "NO" : "#EEEEEE",
        "CC" : "#525100",
        "ECM": "magenta"
    }
    markersize_map = {
        "CV": 2, "PT": 2,
        "HEP": 0.75,
        "NO": 0.75,
        "CC": 1.25,
        "ECM": 1.25
    }

    return (site_types, sites_states, color_map, markersize_map)


def get_simulation_parameters(model_type="model_1") -> Dict[str, float]:
    """_summary_

    Args:
        model_type (str, optional): model type to implement in the simulation. Defaults to "model_1".

    Returns:
        Dict[str, float]: parameters to be used in the simulation, including
            P_CC_GROW - probability of a cancer cell growing to an adjacent site
            P_HEP_DAMAGED - prbability of a healthy hepatocyte damaged by cancer cells to become apoptotic
            P_HEP_CLEARED - probability of an apoptotic hepatocyte becoming cleared
            P_CC_KILLED - probability of a cancer cell killed by an (implicitly implemented) cytotoxic T cell
    """

    # simulate tumour growth
    P_CC_GROW = 1           # probability of a cancer cell growing to an adjacent site
    P_HEP_DAMAGED = 0.5     # probability of a healthy hepatocyte damaged by cancer cells to become apoptotic
    P_HEP_CLEARED = 0.5     # probability of an apoptotic hepatocyte becoming cleared

    parameters = {}
    parameters['P_CC_GROW'] = P_CC_GROW
    parameters['P_HEP_DAMAGED'] = P_HEP_DAMAGED
    parameters['P_HEP_CLEARED'] = P_HEP_CLEARED

    if model_type=="model_1" or model_type=="model_2": # model_2 = model_1 + fibrosis at peri-central regions

        return parameters

    if model_type=="model_3": # model_3 = model_1 + implicit immune predation
        P_CC_KILLED = 0.5       # probability of cancer cells being killed, by implicit immune predation
        parameters['P_CC_KILLED'] = P_CC_KILLED

        return parameters

    if model_type=="model_4": # model_4 = model_1 + move or grow
        P_CC_GROW = 0.5       # probability of cancer cells growing; if not growing then move
        parameters['P_CC_GROW'] = P_CC_GROW

        return parameters


In [8]:
def init_cell_dictionaries(
    lattice: pd.DataFrame,
    n_cancer_cells_init: int,
    CancerCell: CancerCell,
    Hepatocyte: Hepatocyte
) -> Tuple[Dict, pd.DataFrame]:
    """_summary_

    This function initialises cancer cells in the lattice.

    Args:
        lattice (pd.DataFrame): a DataFrame containing information about site_id, x, y, site_type, cell_id, adjacent_site_ids_str, zonation_type
        n_cancer_cells_init (int): the number of cancer cells to initialise in the lattice
        CancerCell (CancerCell): the CancerCell class used for creating new CancerCell objects
        Hepatocyte (Hepatocyte): the Hepatocyte class used for creating new Hepatocyte objects

    Returns:
        Tuple[Dict, pd.DataFrame]: a Tuple of objects including
            cell_dictionaries - a Dict of Dict containing the CancerCell and Hepatocyte objects
            lattice - a DataFrame containing information following initialisation of CancerCell objects
    """

    # initial configuration of hepatocytes
    dict_of_hepatocytes  = {} # id : Hepatocyte()
    for site_id, x, y, site_type, cell_id, adjacent_site_ids_str, zonation_type in lattice.values:

        if site_type == 2:

            cell_attributes = {
                "cell_id": cell_id, "site_id": site_id,
                "cell_position": (x,y),
                "cell_state": 0,
                "lineage_id": np.nan
            }
            hep = Hepatocyte(cell_attributes=cell_attributes)

            dict_of_hepatocytes.update({site_id: hep})

    # introduce the first cancer cell
    dict_of_cancer_cells = {} # id : CancerCell()

    print("BEFORE: total number of hepatocytes: %d " % len(dict_of_hepatocytes))

    cancer_cell_id = int(lattice.cell_id.max() + 1)
    site_ids_to_sample = [
        site_id
        for site_id in dict_of_hepatocytes.keys()
        # if site_id < 100
    ]
    # print(site_ids_to_sample)

    cancer_cell_site_ids = np.random.choice(site_ids_to_sample, size=n_cancer_cells_init, replace=False)
    print(f"> selecting {cancer_cell_site_ids.size} sites to create the first CancerCell objects ")

    for cancer_cell_site_id in cancer_cell_site_ids:

        cancer_cell_xy = dict_of_hepatocytes[cancer_cell_site_id].attributes['cell_position']

        cancer_cell_attributes = {
            "cell_id": cancer_cell_id, "site_id": cancer_cell_site_id,
            "cell_position": cancer_cell_xy,
            "cell_state": 1, # proliferative
            "lineage_id": cancer_cell_id # consider the cancer_cell_id as the lineage-initiating cells
        }

        ## [1] create cancer cell object
        cancer_cell = CancerCell(cell_attributes=cancer_cell_attributes)
        dict_of_cancer_cells.update({cancer_cell_id: cancer_cell})

        ## [2] delete hepatocyte
        del dict_of_hepatocytes[cancer_cell_site_id]

        ## [3] update lattice information
        # print(lattice.loc[lattice.site_id==cancer_cell_site_id])
        lattice.loc[
            lattice.site_id==cancer_cell_site_id,
            ["site_type", "cell_id"]
        ] = (4, cancer_cell_id) # double check cell type corresponds to cancer cell
        # print(lattice.loc[lattice.site_id==cancer_cell_site_id])

        cancer_cell_id += 1

    print("AFTER : total number of hepatocytes : %d " % len(dict_of_hepatocytes))
    print("AFTER : total number of cancer cells: %d " % len(dict_of_cancer_cells))

    cell_dictionaries = {
        "CancerCell": dict_of_cancer_cells,
        "Hepatocyte": dict_of_hepatocytes
    }

    return cell_dictionaries, lattice

In [28]:
def update_cell_states(
    cell_dictionaries: Dict[str, Dict],
    lattice: pd.DataFrame,
    parameters: Dict[str, float],
    CancerCell: CancerCell,
    Hepatocyte: Hepatocyte,
    model_type: str="model_1"
) -> Tuple[Dict, pd.DataFrame]:
    """_summary_

    This function simulates cancer cell proliferation and migration, hepatocyte death, with cell_dictionaries and lattice updated accordingly.

    Args:
        cell_dictionaries (Dict[str, Dict]): a Dict of Dict containing the CancerCell and Hepatocyte objects
        lattice (pd.DataFrame): a DataFrame containing information following initialisation of CancerCell objects
        parameters (Dict[str, float]): parameters to be used in the simulation (see settings.py)
        CancerCell (CancerCell): the CancerCell class used for creating new CancerCell objects
        Hepatocyte (Hepatocyte): the Hepatocyte class used for creating new Hepatocyte objects
        model_type (str, optional): model type to implement in the simulation. Defaults to "model_1".

    Returns:
        Tuple[Dict, pd.DataFrame]: an updated Dict of Dict containing the CancerCell and Hepatocyte objects (with updated attributes; newly created and deleted objects)
    """


    p_cc_grow = parameters['P_CC_GROW']
    p_hep_damaged = parameters['P_HEP_DAMAGED']
    p_hep_cleared = parameters['P_HEP_CLEARED']

    dict_of_cancer_cells, dict_of_hepatocytes = \
        cell_dictionaries['CancerCell'], cell_dictionaries['Hepatocyte']

    new_dict_of_cancer_cells = dict_of_cancer_cells.copy()
    new_dict_of_hepatocytes  = dict_of_hepatocytes.copy()

    # ASSUME for now: only those hepatocytes adjacent to proliferative cancer cells need to be processed
    list_of_hep_ids_to_process = []

    # update states of cancer cells
    for cancer_cell_id, cancer_cell in dict_of_cancer_cells.items():
        # cancer_cell_attributes = cancer_cell.attributes
        cancer_cell_attributes = cancer_cell.get_attributes()

        # cancer_cell_id = cancer_cell_attributes['cell_id']
        cancer_cell_site_id = cancer_cell_attributes['site_id']
        cancer_cell_state = cancer_cell_attributes['cell_state']
        cancer_cell_position = cancer_cell_attributes['cell_position']
        cancer_cell_lineage_id = cancer_cell_attributes['lineage_id']

        # get adjacent site ids
        adjacent_site_ids_str = lattice.loc[
            lattice.site_id==cancer_cell_site_id,
            "adjacent_site_ids_str"
        ].values[0]
        adjacent_site_ids = [
            int(str) for str in adjacent_site_ids_str.split(',')
        ]

        # proliferative cancer cells grow
        if cancer_cell_state == 1:

            # interate over adjacent sites
            for adjacent_site_id in adjacent_site_ids:
                adjacent_site_type, adjacent_site_x, adjacent_site_y, adjacent_site_cell_id = lattice.loc[
                    lattice.site_id==adjacent_site_id,
                    ["site_type", 'x', 'y', 'cell_id']
                ].values[0]

                # grow into NO "Not Occupied" adjacent site
                if adjacent_site_type == 3: # site type = "NO"
                    if np.random.random() < p_cc_grow: # grow to the adjacent site

                        # add a new CancerCell
                        new_cancer_cell_id = int(lattice.cell_id.max() + 1)
                        new_cancer_cell_site_id = adjacent_site_id

                        new_cancer_cell_xy = (adjacent_site_x, adjacent_site_y)

                        new_cancer_cell_attributes = {
                            "cell_id": new_cancer_cell_id, "site_id": new_cancer_cell_site_id,
                            "cell_position": new_cancer_cell_xy,
                            "cell_state": 1, # proliferative,
                            "lineage_id": cancer_cell_lineage_id
                        }

                        ## [1] create cancer cell object
                        new_cancer_cell = CancerCell(cell_attributes=new_cancer_cell_attributes)

                        new_dict_of_cancer_cells |= {new_cancer_cell_id: new_cancer_cell}

                        ## [2] update lattice
                        lattice.loc[
                            lattice.site_id==adjacent_site_id,
                            ["site_type", "cell_id"]
                        ] = (4, new_cancer_cell_id) # sitetype = cancer cell

                    else:
                        if model_type=="model_4": # move to the adjacent site

                            # update the attributes
                            # ... site id
                            # ... xy position
                            cancer_cell_site_id_new = adjacent_site_id
                            cancer_cell_xy_new = (adjacent_site_x, adjacent_site_y)
                            updated_cancer_cell_attributes = {
                                "cell_id": cancer_cell_id, "site_id": cancer_cell_site_id_new,
                                "cell_position": cancer_cell_xy_new,
                                "cell_state": 1, # proliferative
                                "lineage_id": cancer_cell_lineage_id
                            }
                            new_dict_of_cancer_cells[cancer_cell_id].set_attributes(updated_cancer_cell_attributes)

                            # update the lattice site
                            # ... previous site to be emptied
                            # ... new site to be filled
                            lattice.loc[
                                lattice.site_id==cancer_cell_site_id,
                                ["site_type", "cell_id"]
                            ] = (3, np.nan) # sitetype = not occupied
                            lattice.loc[
                                lattice.site_id==cancer_cell_site_id_new,
                                ["site_type", "cell_id"]
                            ] = (4, cancer_cell_id) # sitetype = cancer cell

                elif adjacent_site_type == 2: # site type = "HEP"
                    list_of_hep_ids_to_process.append(adjacent_site_cell_id)

                # (more conditions...)

    # update states of hepatocytes
    # list_of_hep_ids_to_process = list(dict_of_hepatocytes.keys())

    total_number_of_hepatocytes_to_process = len(list_of_hep_ids_to_process)
    number_processed = 0
    # for hep_id, hep in dict_of_hepatocytes.items():
    for hep_id in list_of_hep_ids_to_process:

        if hep_id not in dict_of_hepatocytes.keys() or hep_id not in new_dict_of_hepatocytes.keys():
            continue

        hep = dict_of_hepatocytes[hep_id]

        number_processed += 1
        # if number_processed % (total_number_of_hepatocytes_to_process / 20) == 0:
        #     print(f"> {number_processed / total_number_of_hepatocytes_to_process:.2%} hepatocytes processed...")

        # hep_attributes = hep.attributes
        hep_attributes = hep.get_attributes()

        hep_xy = hep_attributes['cell_position']
        hep_id = hep_attributes['cell_id']
        hep_state = hep_attributes['cell_state']
        hep_site_id = hep_attributes['site_id']

        # get adjacent site ids
        adjacent_site_ids_str = lattice.loc[
            lattice.site_id==hep_site_id,
            "adjacent_site_ids_str"
        ].values[0]
        adjacent_site_ids = [
            int(str) for str in adjacent_site_ids_str.split(',')
        ]

        # quiescent hepatocytes change cell states
        if hep_state == 0:

            # interate over adjacent sites
            for adjacent_site_id in adjacent_site_ids:
                adjacent_site_type, adjacent_site_x, adjacent_site_y = lattice.loc[
                    lattice.site_id==adjacent_site_id,
                    ["site_type", 'x', 'y']
                ].values[0]

                # turn into apoptotic state if a proliferative cancer cell is adjacent
                if adjacent_site_type == 4: # site type = "CC"
                    if np.random.random() < p_hep_damaged:
                        updated_hep_attributes = hep_attributes.copy()
                        updated_hep_attributes['cell_state'] = 2
                        hep.attributes = updated_hep_attributes
                        new_dict_of_hepatocytes[hep_id] = hep

        # apoptotic hepatocytes get cleared
        elif hep_state == 2:
            if np.random.random() < p_hep_cleared: # get cleared

                # [1] delete this hepatocyte
                del new_dict_of_hepatocytes[hep_id]

                # [2] update lattice site type
                lattice.loc[
                    lattice.site_id==hep_site_id,
                    ["site_type", "cell_id"]
                ] = (3, np.nan) # change to Not Occupied

            else: # not get cleared

                if model_type=="model_1": # fibrosis not considered in this model
                    pass

                elif model_type=="model_2": # fibrosis is considered; for simplicity, for apoptotic hepatocytes not cleared, they turn ECM deposited

                    if lattice.loc[
                        lattice.site_id==hep_site_id
                    ].zonation_type.values[0]=="peri-central": # check if this hepatocyte is located in peri-zonal zonation

                        # [1] delete this hepatocyte
                        del new_dict_of_hepatocytes[hep_id]

                        # [2] update lattice site type
                        lattice.loc[
                            lattice.site_id==hep_site_id,
                            ["site_type", "cell_id"]
                        ] = (5, np.nan) # change to ECM

                    # (to be considered) whether or not to introduce ECM as a class

            # (more actions)

        # (more conditions)

    new_cell_dictionaries = {
        "CancerCell": new_dict_of_cancer_cells, "Hepatocyte": new_dict_of_hepatocytes
    }

    return new_cell_dictionaries, lattice

def implicit_immune_predation(
    cell_dictionaries: Dict[str, Dict],
    lattice: pd.DataFrame,
    parameters: Dict[str, float],
    model_type: str="model_3"
) -> Tuple[Dict, pd.DataFrame]:
    """_summary_

    This function simulates cancer cell death (implicitly killed by cytotoxic immune cells), with cell_dictionaries and lattice updated accordingly

    Args:
        cell_dictionaries (Dict[str, Dict]): a Dict of Dict containing the CancerCell and Hepatocyte objects
        lattice (pd.DataFrame): a DataFrame containing information following initialisation of CancerCell objects
        parameters (Dict[str, float]): parameters to be used in the simulation (see settings.py)
        model_type (str, optional): model type to implement in the simulation. Defaults to "model_3".

    Returns:
        Tuple[Dict, pd.DataFrame]: an updated Dict of Dict containing the CancerCell and Hepatocyte objects (with updated attributes; newly created and deleted objects)
    """

    if model_type not in ["model_3"]:
        print("model type is wrong! this function shouldn't be called!")
        return None

    p_cc_killed = parameters['P_CC_KILLED']

    dict_of_cancer_cells, dict_of_hepatocytes = \
        cell_dictionaries['CancerCell'], cell_dictionaries['Hepatocyte']

    new_dict_of_cancer_cells = dict_of_cancer_cells.copy()
    new_dict_of_hepatocytes  = dict_of_hepatocytes.copy()

    n_lattice_sites = lattice.shape[0]
    n_lattice_sites_by_tumour = lattice.loc[lattice.site_type==4].shape[0]
    lattice_tumour = lattice.loc[lattice.site_type==4].copy()

    # randomly sample K * 5 sites as being immune infiltrated/attacked
    lattice_site_ids_immune_attack = np.random.choice(
        lattice.site_id.values,
        size=min(n_lattice_sites_by_tumour * 10, lattice.shape[0]),
        replace=False
        )

    # get the list of cell ids under attack that are tumour
    cancer_cell_ids_to_be_killed = lattice_tumour.loc[
        lattice_tumour.site_id.isin(lattice_site_ids_immune_attack)
    ].cell_id.values

    # kill cancer cells
    for cancer_cell_id in cancer_cell_ids_to_be_killed:
        cancer_cell = dict_of_cancer_cells[cancer_cell_id]
        cancer_cell_attributes = cancer_cell.attributes
        cancer_cell_site_id = cancer_cell_attributes['site_id']
        cancer_cell_state = cancer_cell_attributes['cell_state']
        cancer_cell_position = cancer_cell_attributes['cell_position']
        cancer_cell_lineage_id = cancer_cell_attributes['lineage_id']

        # update p_cc_killed according to lineage_id
        # ... assume cells with even lineage_id is more sensitive, those with odd lineage_id is more resistant

        if cancer_cell_lineage_id % 2 == 0:
            p_cc_killed_heterogeneous = 1
        else:
            p_cc_killed_heterogeneous = 0

        # killed with a probability
        if np.random.random() < p_cc_killed_heterogeneous:

            # [1] delete this cancer cell
            del new_dict_of_cancer_cells[cancer_cell_id]

            # [2] update lattice site type
            lattice.loc[
                lattice.site_id==cancer_cell_site_id,
                ["site_type", "cell_id"]
            ] = (3, np.nan) # change to Not Occupied

    new_cell_dictionaries = {
        "CancerCell": new_dict_of_cancer_cells, "Hepatocyte": new_dict_of_hepatocytes
    }

    return new_cell_dictionaries, lattice

In [10]:
def get_tumour_sizes(
    tumour_t: pd.DataFrame
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """_summary_

    This function detects tumours using the DBSCAN algorithm.

    Args:
        tumour_t (pd.DataFrame): a DataFrame containing information about site_id, x, y, site_type, cell_id, adjacent_site_ids_str, zonation_type

    Returns:
        Tuple[pd.DataFrame, pd.DataFrame]: A Tuple of DataFrame objects including
            tumour_t_sizes - a DataFrame object containing information about DBSCAN cluster ids and sizes
            tumour_t_labelled - a DataFrame object containing information as in tumour_t and further including annotated DBSCAN cluster ids that cells belong to
    """


    dbs = DBSCAN(eps=1.05, min_samples=1) # spacing = 1 in simulation
    dbs.fit(tumour_t[['x','y']].values)
    tumour_t_labelled = tumour_t.copy()
    tumour_t_labelled['label'] = dbs.labels_
    tumour_t_sizes = tumour_t_labelled.groupby('label', as_index=False).agg({'cell_id':'count'})
    tumour_t_sizes.rename(columns={'cell_id': 'size'}, inplace=True)

    return tumour_t_sizes, tumour_t_labelled


## settings

# Main

In [11]:
from google.colab import files
uploaded = files.upload() # choose 'lattice_settings_2025-06-23.json', 'lattice_with_CVs_PTs_2025-06-23_annotated_without_tumour.csv' to upload

Saving lattice_settings_2025-06-23.json to lattice_settings_2025-06-23.json
Saving lattice_with_CVs_PTs_2025-06-23_annotated_without_tumour.csv to lattice_with_CVs_PTs_2025-06-23_annotated_without_tumour.csv


In [12]:
uploaded.keys()

dict_keys(['lattice_settings_2025-06-23.json', 'lattice_with_CVs_PTs_2025-06-23_annotated_without_tumour.csv'])

In [13]:
with open('lattice_settings_2025-06-23.json') as json_file:
    lattice_settings = json.load(json_file)

lattice_without_tumour = pd.read_csv('lattice_with_CVs_PTs_2025-06-23_annotated_without_tumour.csv')

print(lattice_settings)
print(lattice_without_tumour.head(3))

{'spacing': 1, 'lobule_size': 15, 'lattice_size': 60, 'spacing_CV_CV': 25.980762113533157, 'spacing_CV_PT': 15}
   site_id    x         y  site_type  cell_id adjacent_site_ids_str  \
0        0  0.0  0.000000          0        0                   NaN   
1        1  1.0  0.000000          2        1             7,8,2,6,9   
2        2  0.5  0.866025          2        2           8,10,11,3,1   

  zonation_type  
0           NaN  
1  peri-central  
2  peri-central  


## Simulation

In [14]:
# get cell configurations
(site_types, sites_states, color_map, markersize_map) = get_cell_configurations()
print(site_types)

{0: 'CV', 1: 'PT', 2: 'HEP', 3: 'NO', 4: 'CC', 5: 'ECM'}


In [15]:
# parameters

# relevant to all model types
P_CC_GROW = 1       # probability of cancer cells growing
P_HEP_DAMAGED = 0.5 # probability of healthy hepatocytes damaged by cancer cells to become apoptotic
P_HEP_CLEARED = 0.5 # probability of apoptotic hepatocytes becoming cleared

# only relevant to model_3
P_CC_KILLED = 0.5   # probability of cancer cells being killed, by implicit immune predation


parameters = {
    "P_CC_GROW": P_CC_GROW,
    "P_HEP_DAMAGED": P_HEP_DAMAGED,
    "P_HEP_CLEARED": P_HEP_CLEARED,
    "P_CC_KILLED": P_CC_KILLED
}

In [21]:
cancer_cells_seeding_density = 1 # number = seeding density x number of CVs
model_type = "model_3"
T = 20

In [29]:
# simulation

# ...
lattice_in_simulation_without_cancer_cells = lattice_without_tumour.copy()

n_CVs = lattice_in_simulation_without_cancer_cells.loc[
    lattice_in_simulation_without_cancer_cells.site_type==0].shape[0]


# ... initialise cancer cells & create cell dictionaries containing CancerCell and Hepatocyte objects
cell_dictionaries, lattice_in_simulation = init_cell_dictionaries(
    lattice=lattice_in_simulation_without_cancer_cells,
    n_cancer_cells_init=int(cancer_cells_seeding_density * n_CVs),
    CancerCell=CancerCell,
    Hepatocyte=Hepatocyte
)

lattice_in_simulation_copy = lattice_in_simulation.copy()
cell_dictionaries_copy = cell_dictionaries.copy()

snapshots_at_selected_times = pd.DataFrame()
dbscan_clusters_at_selected_times = pd.DataFrame()

for t in np.arange(T+1):

    if t % (T / (T//5)) == 0:

        # total_number_of_cancer_cells = len(cell_dictionaries_copy['CancerCell'])
        # total_number_of_hepatocytes  = len(cell_dictionaries_copy['Hepatocyte'])

        total_number_of_cancer_cells = lattice_in_simulation_copy.loc[lattice_in_simulation_copy.site_type==4].shape[0]
        total_number_of_hepatocytes  = lattice_in_simulation_copy.loc[lattice_in_simulation_copy.site_type==2].shape[0]

        number_of_apoptotic_hepatocytes = len(
            {
                hep_id:hep for hep_id, hep in cell_dictionaries_copy['Hepatocyte'].items()
                if hep.attributes['cell_state'] == 2
            }
        )

        print(f"t = {t}: \n > # of Cancer Cells = {total_number_of_cancer_cells}")
        print(f" > # of Hepatocytes = {total_number_of_hepatocytes}, of which {number_of_apoptotic_hepatocytes} are apoptotic.")

        snapshots_at_t = lattice_in_simulation_copy.copy()
        snapshots_at_t['time'] = t
        snapshots_at_selected_times = pd.concat(
            [snapshots_at_selected_times, snapshots_at_t]
        )

        tumour_t = snapshots_at_t.loc[snapshots_at_t.site_type==4].copy()
        tumour_t_sizes, tumour_t_labelled = get_tumour_sizes(tumour_t=tumour_t)
        tumour_t_labelled['time'] = t
        dbscan_clusters_at_selected_times = pd.concat([dbscan_clusters_at_selected_times, tumour_t_labelled])

    # cancer cell proliferating, damaging hepatocytes
    cell_dictionaries_copy, lattice_in_simulation_copy = update_cell_states(
        cell_dictionaries=cell_dictionaries_copy,
        lattice=lattice_in_simulation_copy,
        parameters=parameters,
        CancerCell=CancerCell,
        Hepatocyte=Hepatocyte,
        model_type=model_type
    )

    # immune cell killing cancer cells
    if model_type=='model_3':
        implicit_immune_predation(
            cell_dictionaries=cell_dictionaries_copy,
            lattice=lattice_in_simulation_copy,
            parameters=parameters,
            model_type=model_type
        )


BEFORE: total number of hepatocytes: 10920 
> selecting 19 sites to create the first CancerCell objects 
AFTER : total number of hepatocytes : 10901 
AFTER : total number of cancer cells: 19 
t = 0: 
 > # of Cancer Cells = 19
 > # of Hepatocytes = 10901, of which 0 are apoptotic.
t = 5: 
 > # of Cancer Cells = 106
 > # of Hepatocytes = 10757, of which 75 are apoptotic.
t = 10: 
 > # of Cancer Cells = 356
 > # of Hepatocytes = 10411, of which 147 are apoptotic.
t = 15: 
 > # of Cancer Cells = 571
 > # of Hepatocytes = 9791, of which 154 are apoptotic.
t = 20: 
 > # of Cancer Cells = 923
 > # of Hepatocytes = 8974, of which 183 are apoptotic.


In [30]:
# visualisation - colour by site type

df_plot = snapshots_at_selected_times.copy()

# ===== scatter plots =====
df_plot["site_type_name"] = [
    site_types[site_type]+"-PC" if zonation_type=='peri-central' and site_type=='HEP' else site_types[site_type]
    for site_type, zonation_type in df_plot[['site_type', 'zonation_type']].values
]
color_map['HEP-PC']='cyan'

sca = px.scatter(
    data_frame=df_plot,
    x='x', y='y',
    color='site_type_name',
    facet_col='time', facet_col_wrap=2,
    color_discrete_map=color_map,
)

# customize the figure
sca.update_layout(
    template='simple_white', width=1000, height=1000
)
sca.update_traces(
    marker=dict(size=3)
)
sca.update_xaxes(title=dict(text="x", font_family="Arial", font_size=14))
sca.update_yaxes(
    title=dict(text="y", font_family="Arial", font_size=14),
    scaleanchor="x", scaleratio=1
    )

In [31]:
# visualisation - colour by dbscan cluster ids

df_plot_2 = dbscan_clusters_at_selected_times.copy()

# ===== scatter plots =====

sca = px.scatter(
    data_frame=df_plot_2,
    x='x', y='y',
    color='label',
    facet_col='time', facet_col_wrap=2,
    color_continuous_scale='HSV',
    hover_data=['label']
)

# customize the figure
sca.update_layout(
    template='simple_white', width=1000, height=1000
)
sca.update_traces(
    marker=dict(size=3)
)
sca.update_xaxes(title=dict(text="x", font_family="Arial", font_size=14))
sca.update_yaxes(
    title=dict(text="y", font_family="Arial", font_size=14),
    scaleanchor="x", scaleratio=1
    )

In [32]:
# visualisation - colour by lineage id

df_plot_3 = dbscan_clusters_at_selected_times.copy()

lineage_ids = []
for cell_id in df_plot_3.cell_id.values:
  cancer_cell = cell_dictionaries_copy['CancerCell'][cell_id]
  lineage_id = cancer_cell.get_attributes()['lineage_id']
  lineage_ids.append(lineage_id)
df_plot_3['lineage_id'] = lineage_ids

# ===== scatter plots =====

sca = px.scatter(
    data_frame=df_plot_3,
    x='x', y='y',
    color='lineage_id',
    facet_col='time', facet_col_wrap=2,
    color_continuous_scale='HSV',
    hover_data=['label', 'lineage_id']
)

# customize the figure
sca.update_layout(
    template='simple_white', width=1000, height=1000
)
sca.update_traces(
    marker=dict(size=3)
)
sca.update_xaxes(title=dict(text="x", font_family="Arial", font_size=14))
sca.update_yaxes(
    title=dict(text="y", font_family="Arial", font_size=14),
    scaleanchor="x", scaleratio=1
    )